In [8]:
import docIterators
import pymongo
from pymongo import MongoClient
import numpy as np
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE, MDS
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import collections
import codecs
import json
import random
from bhtsne.bhtsne import TSNE as BHTSNE
import sys

In [2]:
conn=['mongodb://localhost:6666','Cherry','Strawberry']
iterator = docIterators.StrawberryIter(conn,iter_type='VECTORS')
mc = MongoClient(conn[0])[conn[1]][conn[2]]

In [40]:
with codecs.open('journals.json','r',encoding='utf8') as f:
    c=json.load(f)
c=collections.Counter(c)
vector_type='d2v-d'
journo_coll={}
for journal in zip(*c.most_common()[:20])[0]:
    journo_vectors=[]
    curs = mc.find({'journal':journal})
    for rec in curs[:1000]:
        vectors = rec['vectors']
        journo_vectors.append(vectors[vector_type])
    journo_coll[journal]=journo_vectors
    print(journal + ' collected')

Chemistry - A European Journalcollected
Chem. Lett.collected
Phys. Chem. Chem. Phys.collected
Journal of Materials Chemistrycollected
Food Chemistrycollected
Bioorganic & Medicinal Chemistry Letterscollected
Journal of Heterocyclic Chemistrycollected
Chemistry of Heterocyclic Compoundscollected
Int. J. Quantum Chem.collected
ChemInformcollected
J. Polym. Sci. A Polym. Chem.collected
Journal of Radioanalytical and Nuclear Chemistrycollected
European Journal of Organic Chemistrycollected
Bioorganic & Medicinal Chemistrycollected
Anal Bioanal Chemcollected
Eur. J. Inorg. Chem.collected
Journal of Solid State Chemistrycollected
Organic & Biomolecular Chemistrycollected
Environmental Toxicology and Chemistrycollected
Materials Chemistry and Physicscollected


In [42]:
vecs= [a for b in journo_coll.values() for a in b]
#t = TSNE(init='pca',n_components=2)
t=BHTSNE(n_components=2)
reduced_vecs = t.fit_transform(vecs)
colors=iter(plt.cm.rainbow(np.linspace(0,1,len(journo_coll.keys()))))
start_ind=0
for j in journo_coll.keys():
    no_to_plot=len(journo_coll[j])
    end_ind=start_ind+no_to_plot
    color=colors.next()
    plt.plot(reduced_vecs[start_ind:end_ind,0], reduced_vecs[start_ind:end_ind,1], '.',c=color)
    start_ind=end_ind
    plt.show()